In [1]:
### Data path
#path = "/Users/radleya/The Francis Crick Dropbox/BriscoeJ/Radleya/Claude_Repo/ESFS/Claude_Dev_Folder/"
path = "/Users/radleya/The Francis Crick Dropbox/BriscoeJ/Radleya/Roome_Data/"

### Import ESFS package

In [2]:
import esfs

/opt/anaconda3/envs/Claude_Env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



╔══════════════════════════════════════════════════════════════════════╗
║  ESFS Backend: 🍎 MLX/Metal (Apple Silicon GPU)                 ║
║  Precision: float32                                               ║
╠══════════════════════════════════════════════════════════════════════╣
║  To manually switch backends, run one of:                            ║
║                                                                      ║
║    from esfs import backend                                          ║
║    backend.configure(gpu=False)           # Force CPU                ║
║    backend.configure(gpu=True)            # Auto-detect GPU          ║
║    backend.use_cpu()                      # Force CPU (NumPy/Numba)  ║
║    backend.use_mlx()                      # Force MLX (Apple Silicon)║
║    backend.use_gpu()                      # Force CUDA (NVIDIA)      ║
║                                                                      ║
║  For higher precision (uses more memory):                

### Import additional packages

In [3]:
import plotly.express as px
from scipy.sparse import csc_matrix
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import FuncNorm
from matplotlib.cm import get_cmap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import scanpy as sc

### Load data

In [4]:
#adata = sc.read_h5ad(path+"Claude_Test_Data.h5ad")
adata = sc.read_h5ad(path+"Qiu2024_spinal_E110_E1675.h5ad")


In [5]:
adata

AnnData object with n_obs × n_vars = 297163 × 24552
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell_id', 'experimental_batch', 'major_trajectory', 'celltype_update', 'RNA_snn_res.2', 'seurat_clusters', 'RNA_snn_res.4', 'RNA_snn_res.8', 'dataset', 'Qiu_dv', 'predicted.id', 'prediction.score.dILA2.L', 'prediction.score.v2a8', 'prediction.score.dILB6.CZ2', 'prediction.score.v2b5', 'prediction.score.dILA6.1C1', 'prediction.score.dILB6.B1', 'prediction.score.dILA.Immature', 'prediction.score.dILA3.3A1', 'prediction.score.dILB5.E14.5', 'prediction.score.dILA3.3X2', 'prediction.score.Progenitors', 'prediction.score.dI1.LCN', 'prediction.score.dILB6.E1', 'prediction.score.dILA5.3B', 'prediction.score.dILA2.E', 'prediction.score.v1f', 'prediction.score.dI3a', 'prediction.score.dI1f', 'prediction.score.dILA6.2B', 'prediction.score.v0v7', 'prediction.score.dILB.Immature', 'prediction.score.dILA3.2X2', 'prediction.score.v3c', 'prediction.score.dILB3.A', 'prediction.score.dILA6.1B', 'pre

### Create and save a scaled version of the scRNA-seq counts matrix
Create and save a scaled version of the scRNA-seq counts matrix where each genes has their observed expression values clipped if above a percentile threshold (default=97.5) and are then normalised by their maximum values so that all values are between 0-1, which is a requirment for the Entropy Sort Score (ESS) and Error Potential (EP) calcualtions.

In [6]:
adata = esfs.create_scaled_matrix(adata)

5736 genes show no expression. Removing them from adata object


/Users/radleya/The Francis Crick Dropbox/BriscoeJ/Radleya/Claude_Repo/ESFS/src/esfs/ESFS.py:102: ImplicitModificationWarning: Setting element `.layers['Scaled_Counts']` of view, initializing view as actual.
  adata.layers["Scaled_Counts"] = scaled_expressions.astype(xp.float32)


### Entropy Sorting Gene Set Selection (ES-GSS)

The following step use ES derived functions to perform feature selection that focuses downstream analysis on biologically relevent gene expression profiles.

## Calculate ESS and EP matricies
Here we use the theory of Entropy Sorting to generate a pairwise gene correlation matrix (ESS matrix) and a correlation significance matrix (EP matrix). To speed up calculations parallel processing is implemented. To control the number of cores used for processing, vary the "Use_Cores" parameter which deaults to "Use_Cores=-1" which indicates the software will use n-1 cores, where n is the number of cores available on your machine. 
Please note that "Use_Cores=-1" is as special flag and "Use_Cores=-2" will not use n-2 cores. You should instead change "Use_Cores" to positive integer values of the number of cores you would like to use.

In [ ]:
adata = esfs.parallel_calc_es_matrices(adata, secondary_features_label="Self", save_matrices=np.array(["ESSs","EPs"]), use_cores=14)

Compute: MLX/Metal (Apple Silicon GPU)
Calculating ESS and EP matricies.
If progress bar freezes consider increasing system memory or reducing number of cores used with the 'use_cores' parameter as you may have hit a memory ceiling for your machine.


Processing features:  40%|███▉      | 7520/18816 [06:30<09:27, 19.90genes/s]

esfs.parallel_calc_es_matrices() produces two main outputs which must stay the same as we develop the code, adata.varm['Self_ESSs'] and adata.varm['Self_EPs']

In [ ]:
Output1 = adata.varm['Self_ESSs'] # A pairwise feature correlation matrix
Output2 = adata.varm['Self_EPs'] # A pairwise correlation significance matrix